In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('criminal.csv')

In [12]:
df.columns

Index(['article_id', 'crime', 'penalty'], dtype='object')

In [13]:
# Funkcja do dopasowania opisu przestępstwa
def dopasuj_przestepstwo(wykroczenie):
    # Tworzenie wektorów TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['crime'])
    
    # Wektor wejściowy (wykroczenie)
    wykroczenie_vector = tfidf_vectorizer.transform([wykroczenie])
    
    # Obliczanie podobieństwa kosinusowego
    similarity = cosine_similarity(wykroczenie_vector, tfidf_matrix)
    
    # Znalezienie indeksu najlepszego dopasowania
    best_match_index = similarity.argsort()[0][-1]
    
    # Zwracanie najlepszego dopasowania oraz kary
    return df.iloc[best_match_index]['crime'], df.iloc[best_match_index]['penalty']

In [16]:
# Przykład użycia
wykroczenie_input = "mordowałem"
opis, kara = dopasuj_przestepstwo(wykroczenie_input)

In [17]:
print(f"Dopasowane przestępstwo: {opis}")
print(f"Grożąca kara: {kara}")

Dopasowane przestępstwo: Kto, naruszając prawo międzynarodowe, powoduje u osób:
1) które składając broń lub nie dysponując środkami obrony poddały się,
2) rannych, chorych, rozbitków, personelu medycznego lub osób duchownych,
3) jeńców wojennych,
4) ludności cywilnej obszaru okupowanego, zajętego lub na którym toczą się
działania zbrojne, albo innych osób korzystających w czasie działań zbrojnych
z ochrony międzynarodowej, ciężki uszczerbek na zdrowiu, poddaje te osoby torturom, okrutnemu lub
nieludzkiemu traktowaniu, dokonuje na nich, nawet za ich zgodą, eksperymentów
poznawczych, używa ich do ochraniania swoją obecnością określonego terenu lub
obiektu przed działaniami zbrojnymi albo własnych oddziałów lub zatrzymuje jako
zakładników
Grożąca kara: od 5 lat do 25 lat
